In [81]:
import cv2
import numpy as np
from pycocotools.coco import COCO
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from PIL import Image
import random

## Train dataset Up-sampling - Mosaic

In [82]:
import json
train_json = 'train2_swj_battery.json'
with open(train_json, 'r') as f:
    data = json.load(f)
    info = data['info']
    licenses = data['licenses']
    images = data['images']
    categories = data['categories']
    annotations = data['annotations']

In [83]:
image_ids = []
image_ids2 = []
# 적은 클래스에 대해서 mosaic
# [2: "Paper pack", 3: "Metal", 4: "Glass", 6: "Styrofoam", 8: "Battery", 9: "Clothing"]
coco = COCO(train_json)
for ann in annotations:
     if ann['category_id'] in [2, 3, 4, 6, 8, 9]:
        image_ids.append(ann['image_id'])
        image_id = coco.getImgIds(imgIds=ann['image_id'])
        image_info = coco.loadImgs(image_id)[0]
        cat = [x['category_id'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        count = 0
        for c in cat:
            if c not in [0, 1, 5, 7]:
                count += 1
            if count == len(cat):
                image_ids2.append(ann['image_id'])

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


In [84]:
print(len(image_ids))
image_ids = list(set(image_ids))
print(len(image_ids)) # 중복 제거한 image 갯수

5138
2048


In [85]:
print(len(image_ids2))
image_ids2 = list(set(image_ids2))
print(len(image_ids2)) # 많은 클래스도 포함하지 않은 image 갯수

1576
790


In [86]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

folder_dir = 'mosaic_aug2'
if not os.path.exists(folder_dir):
    os.makedirs(folder_dir)

# mosaic train - 100000 부터 시작
image_num = 100000
annotation_num = 100000
coco = COCO(train_json)

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

for n in range(500): # mosaic 이미지를 몇개 만들지 (500개)
    # 랜덤하게 4개 이미지 고르기
    select_image_id = random.choices(image_ids2, k=4)
    random.shuffle(select_image_id)
    print('======== ', n ,' =========')
    print(select_image_id)
    
    # 중심좌표
    mosaic_border = [-1024 // 2, -1024 // 2]
    yc, xc = (int(random.uniform(-x, 2 * 1024 + x)) for x in mosaic_border)
    
    # mosaic할 이미지 보드판, 114로 채움(회색)
    mosaic_img = np.full((1024*2, 1024*2, 3), 114, dtype=np.uint8)
    
    new_annotations = []

    # 보드판에 중심점을 기준으로 왼쪽위, 오른쪽위, 왼쪽아래, 오른쪽아래 붙이기
    for i, id in enumerate(select_image_id):
        print('current image_id :', id)
        # id = select_image_id[0]
        image_id = coco.getImgIds(imgIds=id)
        image_info = coco.loadImgs(image_id)[0]

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        img = cv2.imread(image_info['file_name'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        h, w = img.shape[:2]
        s = 1024
        max_w_h = 2048

        if i == 0:
            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            xmin, ymin = x1a, y1a
        elif i == 1:
            x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
            x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
        elif i == 2:
            x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, w, min(y2a - y1a, h)
        elif i == 3:
            x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            xmax, ymax = x2a, y2a

        for (bx, by, bw, bh), c in zip(box, cat):
            if i == 0:
                x, y = max(xc-w+bx, 0), max(yc-h+by, 0)
                if x+bw > xc:
                    bw = bw-(bw-xc)
                if y+bh > yc:
                    bh = bh-(bh-yc)
            elif i == 1:
                x, y = xc+bx, max(yc-h+by, 0)
                if x+bw > max_w_h:
                    bw = bw-(bw-max_w_h)
                if y+bh > yc:
                    bh = bh-(bh-yc)
            elif i == 2:
                x, y = max(xc-w+bx, 0), yc+by
                if x+bw > xc:
                    bw = bw-(bw-xc)
                if y+bh > max_w_h:
                    bh = bh-(bh-max_w_h)
            elif i == 3:
                x, y = xc+bx, yc+by
                if x+bw > max_w_h:
                    bw = bw-(bw-max_w_h)
                if y+bh > max_w_h:
                    bh = bh-(bh-max_w_h)

            if (x >= 0 and x < max_w_h) and (y >= 0 and y < max_w_h) and (bw > 0 and bh > 0):
                new_annotations.append({
                    'image_id': image_num,
                    'category_id': classes.index(c),
                    'area': round(bw*bh, 2),
                    'bbox': [round(x, 1), round(y, 1), round(bw, 1), round(bh, 1)],
                    'iscrowd': 0,
                    'id': annotation_num})
                annotation_num += 1

        mosaic_img[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]
    
    # 빈부분제외해서 잘라내기
    mosaic_img = mosaic_img[ymin:ymax, xmin:xmax,:] 
    
    # mosaic_img 크기를 1024x1024로 resize
    w_scale = 1024/mosaic_img.shape[1]
    h_scale = 1024/mosaic_img.shape[0]
    for ann in new_annotations:
        ann['bbox'][0] = round((ann['bbox'][0]-xmin)*w_scale, 1)
        ann['bbox'][1] = round((ann['bbox'][1]-ymin)*h_scale, 1)
        ann['bbox'][2] = round((ann['bbox'][2])*w_scale, 1)
        ann['bbox'][3] = round((ann['bbox'][3])*h_scale, 1)
        ann['area'] = round(ann['bbox'][2]*ann['bbox'][3], 2)
        annotations.append(ann)
    
    mosaic_img = cv2.resize(mosaic_img, (1024, 1024))
    images.append({
            'width': mosaic_img.shape[0],
            'height': mosaic_img.shape[1],
            'file_name': folder_dir+'/'+str(image_num)+'.jpg',
            'license': 0,
            'flickr_url': None,
            'coco_url': None,
            'date_captured': None,
            'id': image_num
        })
    mosaic_img = Image.fromarray((mosaic_img.astype(np.uint8)))
    mosaic_img.save(folder_dir+'/'+str(image_num)+'.jpg')
    image_num += 1

# json 파일 저장
train_mosaic_json = 'train2_swj_battery_mosaic2.json'
with open(train_mosaic_json, 'w') as f:
    json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
            'annotations': annotations, 'categories': categories}, f)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
========  0  =========
[4950, 2218, 2844, 4649]
current image_id : 4950
current image_id : 2218
current image_id : 2844
current image_id : 4649
========  1  =========
[2684, 4977, 495, 467]
current image_id : 2684
current image_id : 4977
current image_id : 495
current image_id : 467
========  2  =========
[2069, 4948, 2051, 717]
current image_id : 2069
current image_id : 4948
current image_id : 2051
current image_id : 717
========  3  =========
[3526, 3309, 4385, 1946]
current image_id : 3526
current image_id : 3309
current image_id : 4385
current image_id : 1946
========  4  =========
[2187, 4996, 1544, 2501]
current image_id : 2187
current image_id : 4996
current image_id : 1544
current image_id : 2501
========  5  =========
[4888, 3676, 3968, 2386]
current image_id : 4888
current image_id : 3676
current image_id : 3968
current image_id : 2386
========  6  =========
[2651, 5033, 1501, 2203]
current ima

## Validation dataset Up-sampling - Mosaic

In [87]:
import json
train_json = 'val2_swj_battery.json'
with open(train_json, 'r') as f:
    data = json.load(f)
    info = data['info']
    licenses = data['licenses']
    images = data['images']
    categories = data['categories']
    annotations = data['annotations']

In [88]:
image_ids = []
image_ids2 = []
# 적은 클래스에 대해서 mosaic
# [2: "Paper pack", 3: "Metal", 4: "Glass", 6: "Styrofoam", 8: "Battery", 9: "Clothing"]
coco = COCO(train_json)
for ann in annotations:
     if ann['category_id'] in [2, 3, 4, 6, 8, 9]:
        image_ids.append(ann['image_id'])
        image_id = coco.getImgIds(imgIds=ann['image_id'])
        image_info = coco.loadImgs(image_id)[0]
        cat = [x['category_id'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        count = 0
        for c in cat:
            if c not in [0, 1, 5, 7]:
                count += 1
            if count == len(cat):
                image_ids2.append(ann['image_id'])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [89]:
print(len(image_ids))
image_ids = list(set(image_ids))
print(len(image_ids)) # 중복 제거한 image 갯수

396
250


In [90]:
print(len(image_ids2))
image_ids2 = list(set(image_ids2))
print(len(image_ids2)) # 중복 제거한 image 갯수

201
175


In [91]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# mosaic val - 110000 부터 시작
image_num = 110000
annotation_num = 110000
coco = COCO(train_json)

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

for n in range(100): # mosaic 이미지를 몇개 만들지 (100개)
    # 랜덤하게 4개 이미지 고르기
    select_image_id = random.choices(image_ids2, k=4)
    random.shuffle(select_image_id)
    print('======== ', n ,' =========')
    print(select_image_id)
    
    # 중심좌표
    mosaic_border = [-1024 // 2, -1024 // 2]
    yc, xc = (int(random.uniform(-x, 2 * 1024 + x)) for x in mosaic_border)
    
    # mosaic할 이미지 보드판, 114로 채움(회색)
    mosaic_img = np.full((1024*2, 1024*2, 3), 114, dtype=np.uint8)
    
    new_annotations = []

    # 보드판에 중심점을 기준으로 왼쪽위, 오른쪽위, 왼쪽아래, 오른쪽아래 붙이기
    for i, id in enumerate(select_image_id):
        print('current image_id :', id)
        # id = select_image_id[0]
        image_id = coco.getImgIds(imgIds=id)
        image_info = coco.loadImgs(image_id)[0]

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        img = cv2.imread(image_info['file_name'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        h, w = img.shape[:2]
        s = 1024
        max_w_h = 2048

        if i == 0:
            x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            xmin, ymin = x1a, y1a
        elif i == 1:
            x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
            x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
        elif i == 2:
            x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, w, min(y2a - y1a, h)
        elif i == 3:
            x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
            x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            xmax, ymax = x2a, y2a

        for (bx, by, bw, bh), c in zip(box, cat):
            if i == 0:
                x, y = max(xc-w+bx, 0), max(yc-h+by, 0)
                if x+bw > xc:
                    bw = bw-(bw-xc)
                if y+bh > yc:
                    bh = bh-(bh-yc)
            elif i == 1:
                x, y = xc+bx, max(yc-h+by, 0)
                if x+bw > max_w_h:
                    bw = bw-(bw-max_w_h)
                if y+bh > yc:
                    bh = bh-(bh-yc)
            elif i == 2:
                x, y = max(xc-w+bx, 0), yc+by
                if x+bw > xc:
                    bw = bw-(bw-xc)
                if y+bh > max_w_h:
                    bh = bh-(bh-max_w_h)
            elif i == 3:
                x, y = xc+bx, yc+by
                if x+bw > max_w_h:
                    bw = bw-(bw-max_w_h)
                if y+bh > max_w_h:
                    bh = bh-(bh-max_w_h)

            if (x >= 0 and x < max_w_h) and (y >= 0 and y < max_w_h) and (bw > 0 and bh > 0):
                new_annotations.append({
                    'image_id': image_num,
                    'category_id': classes.index(c),
                    'area': round(bw*bh, 2),
                    'bbox': [round(x, 1), round(y, 1), round(bw, 1), round(bh, 1)],
                    'iscrowd': 0,
                    'id': annotation_num})
                annotation_num += 1

        mosaic_img[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]
    
    # 빈부분제외해서 잘라내기
    mosaic_img = mosaic_img[ymin:ymax, xmin:xmax,:] 
    
    # mosaic_img 크기를 1024x1024로 resize
    w_scale = 1024/mosaic_img.shape[1]
    h_scale = 1024/mosaic_img.shape[0]
    for ann in new_annotations:
        ann['bbox'][0] = round((ann['bbox'][0]-xmin)*w_scale, 1)
        ann['bbox'][1] = round((ann['bbox'][1]-ymin)*h_scale, 1)
        ann['bbox'][2] = round((ann['bbox'][2])*w_scale, 1)
        ann['bbox'][3] = round((ann['bbox'][3])*h_scale, 1)
        ann['area'] = round(ann['bbox'][2]*ann['bbox'][3], 2)
        annotations.append(ann)
    
    mosaic_img = cv2.resize(mosaic_img, (1024, 1024))
    images.append({
            'width': mosaic_img.shape[0],
            'height': mosaic_img.shape[1],
            'file_name': folder_dir+'/'+str(image_num)+'.jpg',
            'license': 0,
            'flickr_url': None,
            'coco_url': None,
            'date_captured': None,
            'id': image_num
        })
    mosaic_img = Image.fromarray((mosaic_img.astype(np.uint8)))
    mosaic_img.save(folder_dir+"/"+str(image_num)+'.jpg')
    image_num += 1

# json 파일 저장
train_mosaic_json = 'val2_swj_battery_mosaic2.json'
with open(train_mosaic_json, 'w') as f:
    json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
            'annotations': annotations, 'categories': categories}, f)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
========  0  =========
[146, 1141, 4289, 5106]
current image_id : 146
current image_id : 1141
current image_id : 4289
current image_id : 5106
========  1  =========
[1941, 1342, 3198, 4511]
current image_id : 1941
current image_id : 1342
current image_id : 3198
current image_id : 4511
========  2  =========
[4511, 1841, 63, 288]
current image_id : 4511
current image_id : 1841
current image_id : 63
current image_id : 288
========  3  =========
[4394, 434, 2766, 2682]
current image_id : 4394
current image_id : 434
current image_id : 2766
current image_id : 2682
========  4  =========
[5103, 5102, 3783, 2318]
current image_id : 5103
current image_id : 5102
current image_id : 3783
current image_id : 2318
========  5  =========
[288, 4112, 2318, 434]
current image_id : 288
current image_id : 4112
current image_id : 2318
current image_id : 434
========  6  =========
[3028, 5100, 3623, 4343]
current image_id : 

## 이미지 예시 출력해보기

In [90]:
import json
train_json = 'train2_swj_battery.json'
with open(train_json, 'r') as f:
    data = json.load(f)
    info = data['info']
    licenses = data['licenses']
    images = data['images']
    categories = data['categories']
    annotations = data['annotations']

In [91]:
image_ids = []
# 적은 클래스에 대해서 mosaic
# [2: "Paper pack", 3: "Metal", 4: "Glass", 6: "Styrofoam", 8: "Battery", 9: "Clothing"]
for ann in annotations:
     if ann['category_id'] in [2, 3, 4, 6, 8, 9]:
        image_ids.append(ann['image_id'])

In [92]:
select_image_id = random.choices(image_ids, k=4)
random.shuffle(select_image_id)
print(select_image_id)

[5003, 1996, 3229, 2539]


In [93]:
mosaic_border = [-1024 // 2, -1024 // 2]
yc, xc = (int(random.uniform(-x, 2 * 1024 + x)) for x in mosaic_border) 

In [94]:
yc, xc  # 중심 좌표

(972, 824)

In [95]:
# mosaic할 이미지 보드판, 114로 채움(회색)
mosaic_img = np.full((1024*2, 1024*2, 3), 114, dtype=np.uint8)

In [ ]:
Image.fromarray((mosaic_img).astype(np.uint8))

In [ ]:
import os
import cv2
import numpy as np
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
coco = COCO(train_json)

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

n = select_image_id[1]

image_id = coco.getImgIds(imgIds=n)
image_info = coco.loadImgs(image_id)[0]

box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

img = cv2.imread(image_info['file_name'])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
img /= 255

fig, ax = plt.subplots(figsize=(4, 4))
ax.set_title('image_id:'+str(n))
plt.xticks([])
plt.yticks([])
for (x, y, w, h), c in zip(box, cat):
    label_color = LABELCOLORS[classes.index(c)]
    ax.add_patch(patches.Rectangle((x, y), w, h, edgecolor=label_color, linewidth=1, fill=False))
    ax.text(x, y-5, c, fontsize=9, color=label_color)
plt.imshow(img)

In [ ]:
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

coco = COCO(train_json)

fig, ax = plt.subplots(figsize=(12, 12))

new_annotations = []
new_box = []
image_num = 0
annotation_num = 0

for i, id in enumerate(select_image_id):
    print('current image_id :', id)
    # id = select_image_id[0]
    image_id = coco.getImgIds(imgIds=id)
    image_info = coco.loadImgs(image_id)[0]

    box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
    cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

    img = cv2.imread(image_info['file_name'])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)

    box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
    cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
    
    h, w = img.shape[:2]
    s = 1024
    max_w_h = 2048
    
    if i == 0:
        x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
        x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
        xmin, ymin = x1a, y1a
    elif i == 1:
        x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
        x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
    elif i == 2:
        x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
        x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, w, min(y2a - y1a, h)
    elif i == 3:
        x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
        x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
        xmax, ymax = x2a, y2a

    for (bx, by, bw, bh), c in zip(box, cat):
        label_color = LABELCOLORS[classes.index(c)]
        if i == 0:
            x, y = max(xc-w+bx, 0), max(yc-h+by, 0)
            if x+bw > xc:
                bw = bw-(bw-xc)
            if y+bh > yc:
                bh = bh-(bh-yc)
        elif i == 1:
            x, y = xc+bx, max(yc-h+by, 0)
            if x+bw > max_w_h:
                bw = bw-(bw-max_w_h)
            if y+bh > yc:
                bh = bh-(bh-yc)
        elif i == 2:
            x, y = max(xc-w+bx, 0), yc+by
            if x+bw > xc:
                bw = bw-(bw-xc)
            if y+bh > max_w_h:
                bh = bh-(bh-max_w_h)
        elif i == 3:
            x, y = xc+bx, yc+by
            if x+bw > max_w_h:
                bw = bw-(bw-max_w_h)
            if y+bh > max_w_h:
                bh = bh-(bh-max_w_h)

        if (x >= 0 and x < max_w_h) and (y >= 0 and y < max_w_h) and (bw > 0 and bh > 0):
            annotation_num += 1
            new_box.append([round(x, 1), round(y, 1), round(bw, 1), round(bh, 1)])
            new_annotations.append({
                'image_id': image_num,
                'category_id': classes.index(c),
                'area': round(bw*bh, 2),
                'bbox': [round(x, 1), round(y, 1), round(bw, 1), round(bh, 1)],
                'iscrowd': 0,
                'id': annotation_num})
            ax.add_patch(patches.Rectangle((x, y), bw, bh, edgecolor=label_color, linewidth=1, fill=False))
            ax.text(x, y-5, c, fontsize=9, color=label_color)

    mosaic_img[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]

print(new_annotations)
plt.xticks([])
plt.yticks([])
ax.imshow(mosaic_img)

In [ ]:
mosaic_img = mosaic_img[ymin:ymax, xmin:xmax,:] # 빈부분제외해서 잘라내기
print(mosaic_img.shape)
Image.fromarray((mosaic_img.astype(np.uint8)))

In [ ]:
# mosaic_img 크기를 1024x1024로 resize
w_scale = 1024/mosaic_img.shape[1]
h_scale = 1024/mosaic_img.shape[0]
for ann in new_annotations:
    ann['bbox'][0] = round((ann['bbox'][0]-xmin)*w_scale, 1)
    ann['bbox'][1] = round((ann['bbox'][1]-ymin)*h_scale, 1)
    ann['bbox'][2] = round((ann['bbox'][2])*w_scale, 1)
    ann['bbox'][3] = round((ann['bbox'][3])*h_scale, 1)
    ann['area'] = round(ann['bbox'][2]*ann['bbox'][3], 2)

fig, ax = plt.subplots(figsize=(12, 12))
plt.xticks([])
plt.yticks([])

box = [ann['bbox'] for ann in new_annotations]
cat = [ann['category_id'] for ann in new_annotations]

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

mosaic_img = cv2.resize(mosaic_img, (1024, 1024))
for (x, y, w, h), c in zip(box, cat):
    label_color = LABELCOLORS[c]
    ax.add_patch(patches.Rectangle((x, y), w, h, edgecolor=label_color, linewidth=1, fill=False))
    ax.text(x, y-5, classes[c], fontsize=9, color=label_color)
plt.imshow(mosaic_img)
# mosaic_img = Image.fromarray((mosaic_img.astype(np.uint8)))
# mosaic_img.save("mosaic_aug/"+str(image_num)+'.jpg')